# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

D. M. Fuksman  ->  D. M. Fuksman  |  ['D. M. Fuksman']
M. Flock  ->  M. Flock  |  ['M. Flock']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
G. Mattia  ->  G. Mattia  |  ['G. Mattia']
D. Muley  ->  D. Muley  |  ['D. Muley']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Shi  ->  J. Shi  |  ['J. Shi']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 82 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.13999


extracting tarball to tmp_2504.13999...

 done.


D. M. Fuksman  ->  D. M. Fuksman  |  ['D. M. Fuksman']
M. Flock  ->  M. Flock  |  ['M. Flock']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
G. Mattia  ->  G. Mattia  |  ['G. Mattia']
D. Muley  ->  D. Muley  |  ['D. Muley']


...

 done.
  0: tmp_2504.14180/main.tex, 1,052 lines
  1: tmp_2504.14180/reference.tex, 8 lines
Retrieving document from  https://arxiv.org/e-print/2504.14836


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2504.14180/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'main.bbl' from 'tmp_2504.14180/main.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2504.14836...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.15086


extracting tarball to tmp_2504.15086... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.13999-b31b1b.svg)](https://arxiv.org/abs/2504.13999) | **Multidimensional half-moment multigroup radiative transfer. Improving moment-based thermal models of circumstellar disks**  |
|| <mark>D. M. Fuksman</mark>, <mark>M. Flock</mark>, <mark>H. Klahr</mark>, <mark>G. Mattia</mark>, <mark>D. Muley</mark> |
|*Appeared on*| *2025-04-22*|
|*Comments*| *Submitted to Astronomy & Astrophysics, comments welcome*|
|**Abstract**|            Common radiative transfer methods, such as flux-limited diffusion (FLD) and the M1 closure, suffer from artificial interactions between crossing beams. In protoplanetary disks, this leads to an overestimation of the midplane temperature due to the merging of vertical inward and outward fluxes. Methods that avoid these artifacts typically require angular discretization, which can be computationally expensive. In the spirit of the two-stream approximation, we aim to remove the interaction between beams in a fixed spatial direction by introducing a half-moment (HM) closure, which integrates the radiative intensity over hemispheres. We derive a multidimensional HM closure via entropy maximization and replace it with an approximate expression that closely matches it, coinciding in the diffusion and free-streaming regimes while remaining expressible through simple operations. We implement the HM and M1 closures via implicit-explicit (IMEX) schemes, including multiple frequency groups. We test these methods in numerical benchmarks, including computing the temperature in an irradiated disk around a T Tauri star, comparing the results with Monte Carlo (MC) radiative transfer simulations. The resulting HM closure tends to the correct limit in the diffusion regime and prevents interactions between crossing fluxes in a chosen spatial direction. In disk simulations with 22 frequency groups, the M1 closure disagrees with the MC midplane temperature by up to 21%, while HM reduces this discrepancy to 6%. Even with just 3 frequency groups, HM significantly outperforms M1, with maximum departures of 8% compared to M1's 23%.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.14180-b31b1b.svg)](https://arxiv.org/abs/2504.14180) | **Deep Neural Networks for Modeling Astrophysical Nuclear reacting flows**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-04-22*|
|*Comments*| **|
|**Abstract**|            In astrophysical simulations, nuclear reacting flows pose computational challenges due to the stiffness of reaction networks. We introduce neural network-based surrogate models using the DeePODE framework to enhance simulation efficiency while maintaining accuracy and robustness. Our method replaces conventional stiff ODE solvers with deep learning models trained through evolutionary Monte Carlo sampling from zero-dimensional simulation data, ensuring generalization across varied thermonuclear and hydrodynamic conditions. Tested on 3-species and 13-species reaction networks, the models achieve $\lesssim 1\%$ accuracy relative to semi-implicit numerical solutions and deliver a $\sim 2.6\times$ speedup on CPUs. A temperature-thresholded deployment strategy ensures stability in extreme conditions, sustaining neural network utilization above 75\% in multi-dimensional simulations. These data-driven surrogates effectively mitigate stiffness constraints, offering a scalable approach for high-fidelity modeling of astrophysical nuclear reacting flows.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.14836-b31b1b.svg)](https://arxiv.org/abs/2504.14836) | **Systematic search for blue hyper-velocity stars from LAMOST survey**  |
|| Y. Sun, et al. -- incl., <mark>J. Liu</mark>, <mark>J. Shi</mark> |
|*Appeared on*| *2025-04-22*|
|*Comments*| *21 pages, 11 figures, accepted for publication in APJ*|
|**Abstract**|            Hypervelocity stars (HVSs) represent a unique class of objects capable of escaping the gravitational pull of the Milky Way due to extreme acceleration events, such as close encounters with the supermassive black hole at the Galactic center (GC), supernova explosions in binary systems, or multi-body dynamical interactions. Finding and studying HVSs are crucial to exploring these ejection mechanisms, characterizing central black holes, probing the GC environment, and revealing the distribution of dark matter in our galaxy. The Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST) spectroscopic surveys have so far identified four B-type unbound HVSs. To expand this sample with the second-phase LAMOST survey that started in 2018, we conducted a systematic search for early-type HVSs using the LAMOST Data Release 10. We identified 125 early-type high-velocity candidates with total velocities exceeding 300 km\,s$^{-1}$. Among them, we report ten new unbound B- and A-type hypervelocity star (HVS) candidates (designated LAMOST-HVS5 through LAMOST-HVS14), tripling the number of unbound HVSs previously identified by LAMOST. Kinematic analyses suggest that these newly discovered HVS candidates likely originated either from the Galactic Center or via dynamical interactions. Future high-resolution follow-up observations promise to refine the stellar parameters, distances, and elemental abundances of these candidates, thereby providing deeper insights into their origins and broadening their potential applications across astrophysics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.15086-b31b1b.svg)](https://arxiv.org/abs/2504.15086) | **Configuration Requirements for 21-cm Forest Background Quasar Searches with the Moon-based Interferometer**  |
|| S. Zhang, Q. Niu, Y. Li, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-22*|
|*Comments*| *8 pages, 5 figures*|
|**Abstract**|            The 21-cm forest offers a powerful cosmological probe of the thermal history and small-scale structure of the intergalactic medium during the Epoch of Reionization (EoR). Its success, however, critically depends on the availability of high-redshift radio-loud quasars (HzRLQs) as background sources. In this work, we investigate the configuration requirements for a Moon-based low-frequency radio interferometer aimed at maximizing the detection of HzRLQs for future 21-cm forest studies. Building upon a previously developed quasar luminosity function (QLF), we forecast HzRLQ abundances under various array configurations. Assuming a total survey area of $10^4\,\mathrm{deg}^2$ and 1 year of observation, we compare continuum surveys with 10 MHz bandwidth and 21-cm forest surveys with 5 kHz resolution. Our results show that a minimum collecting area of $\sim$6 500 m$^2$ enables detection at $z \sim 6$, while SKA-like arrays ($N_{\mathrm{st}} = 512$) extend the detection limit to $z \sim 10$ for 21-cm forest survey and $z \sim 16$ for continuum survey. Larger arrays with $N_{\mathrm{st}} = 2048$ can reach $z \sim 11$ in 21-cm forest mode. We also explore configurations that maintain fixed collecting areas while increasing the number to enhance survey efficiency. This boosts source detection but significantly increases the data volume and computational demands. These results underscore the importance of optimizing array design for different survey goals and balancing sensitivity, spectral resolution, and data management. A well-designed Moon-based array could open a new observational window on reionization and early cosmic structure formation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.13999/./beams_combined.png', 'tmp_2504.13999/./shadow_limiting.png', 'tmp_2504.13999/./Disk_FpEp_v2.png']
copying  tmp_2504.13999/./beams_combined.png to _build/html/
copying  tmp_2504.13999/./shadow_limiting.png to _build/html/
copying  tmp_2504.13999/./Disk_FpEp_v2.png to _build/html/
exported in  _build/html/2504.13999.md
    + _build/html/tmp_2504.13999/./beams_combined.png
    + _build/html/tmp_2504.13999/./shadow_limiting.png
    + _build/html/tmp_2504.13999/./Disk_FpEp_v2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sftw}[1]{\texttt{#1}}$
$\newcommand{\RED}{\color{red}}$
$\newcommand{\BLACK}{\color{black}}$
$\newcommand{\BLUE}{\color{blue}}$
$\newcommand{\GREEN}{\color{green1}}$
$\newcommand{\DS}{\displaystyle}$
$\newcommand{\cF}{{\cal F}}$
$\newcommand{\cG}{{\cal G}}$
$\newcommand{\cR}{{\cal R}}$
$\newcommand{\cS}{{\cal S}}$
$\newcommand{\cU}{{\cal U}}$
$\newcommand{\cV}{{\cal V}}$
$\newcommand{\norm}[1]{ {\left|\left|{#1}\right|\right|}}$
$\newcommand{\dmnote}[1]{{\bf \color{blue}[D.M.: #1]}}$
$\newcommand\blue{#1}$
$\newcommand\green{#1}$
$\newcommand\red{#1}$</div>



<div id="title">

# Multidimensional half-moment multigroup radiative transfer: Improving moment-based thermal models of circumstellar disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.13999-b31b1b.svg)](https://arxiv.org/abs/2504.13999)<mark>Appeared on: 2025-04-22</mark> -  _Submitted to Astronomy & Astrophysics, comments welcome_

</div>
<div id="authors">

<mark>D. M. Fuksman</mark>, <mark>M. Flock</mark>, <mark>H. Klahr</mark>, <mark>G. Mattia</mark>, <mark>D. Muley</mark>

</div>
<div id="abstract">

**Abstract:** Common radiative transfer methods, such as flux-limited diffusion (FLD) and the M1 closure, suffer from artificial interactions between crossing beams. In protoplanetary disks, this leads to an overestimation of the midplane temperature due to the merging of vertical inward and outward fluxes. Methods that avoid these artifacts typically require angular discretization, which can be computationally expensive. In the spirit of the two-stream approximation, we aim to remove the interaction between beams in a fixed spatial direction by introducing a half-moment (HM) closure, which integrates the radiative intensity over hemispheres. We derived a multidimensional HM closure via entropy maximization and replaced this closure with an approximate expression that closely matches it, coinciding in the diffusion and free-streaming regimes while remaining expressible through simple operations.We implementedthe HM and M1 closures via implicit-explicit (IMEX) schemes,  including multiple frequency groups.We testedthese methods in numerical benchmarks, including computing the temperature in an irradiateddisk around a T Tauri star, comparing the results with Monte Carlo (MC) radiative transfer simulations. The resulting HM closure tends to the correct limit in the diffusion regime and prevents interactions between crossing fluxes in a chosen spatial direction.In disk simulations with 22 frequency groups, the M1 closure disagrees with the MC midplane temperature by up to $21\%$ , while HM reduces this discrepancy to $6\%$ . Even with just 3 frequency groups, HM significantly outperforms M1, with maximum departures of $8\%$ compared to M1's $23\%$ .

</div>

<div id="div_fig1">

<img src="tmp_2504.13999/./beams_combined.png" alt="Fig2" width="100%"/>

**Figure 2. -** Crossing beams advected with the M1 and HM closures. This test highlights how the HM closure removes the artificial interaction between crossing fluxes in selected directions. In the HM case, $\hat{\mathbf{d}}=\hat{\mathbf{x}}$ has been chosen as the splitting direction. Arrows indicate the directions of the injected beams.
 (*fig:crossing_beams*)

</div>
<div id="div_fig2">

<img src="tmp_2504.13999/./shadow_limiting.png" alt="Fig6" width="100%"/>

**Figure 6. -** Total energy density in the horizontal shadow test obtained with the M1 and HM closures. For the HM closure, distributions are shown for the cases where Eq. \eqref{Eq:physlimit2} is not enforced, enforced via Eq. \eqref{Eq:Fminv1}, and enforced via Eq. \eqref{Eq:Fminv2}, in the second, third, and fourth panels from the top, respectively.
 (*fig:shadow*)

</div>
<div id="div_fig3">

<img src="tmp_2504.13999/./Disk_FpEp_v2.png" alt="Fig12" width="100%"/>

**Figure 12. -** 
Half-moment ($+$) flux and energy density at $r=1$ au for the highest-energy group in the disk tests with $N_g=3$, given in erg cm$^{-3}$. Solid and dotted lines correspond to solutions obtained with and without enforcing Eq. \eqref{Eq:Fminv2}, respectively.
 (*fig:DiskFpEp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.13999"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

449  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
